# RIC and RBDS debugg joint angle calculation

Reginaldo K Fukuchi 

This NB implements the "gait_steps.m" Sean Osis method to detect gait events.

In [1]:
# Prepare environment
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.insert(1, r'../functions')

In [2]:
from ezc3d import c3d
import parse_gait_kinematics as parse_gait

### RIC data

In [3]:
figshare_dir = r'C:\Users\Reginaldo\OneDrive - University of Calgary\data\Figshare_SciData\new_unzip'
fn_json=os.path.join(figshare_dir, '201225', '20140515T133244.json')
neutral_RIC, joints_RIC, gait_RIC, hz_RIC = parse_gait.parse_RIC(fn_json)

Export static to c3d

In [8]:
# Load an empty c3d structure
c3d = c3d()

# Merge neutral and joints prior create static c3d
neutral_RIC = pd.concat([neutral_RIC, joints_RIC], axis=1)

neutral_lbls = list(dict.fromkeys([mkr[:-2] for mkr in neutral_RIC.columns.tolist()]))
# Fill it with random data
c3d['parameters']['POINT']['RATE']['value'] = [hz_RIC]
c3d['parameters']['POINT']['LABELS']['value'] = neutral_lbls

x_mkr = neutral_RIC.values[0,0:-2:3]
y_mkr = neutral_RIC.values[0,1:-1:3]
z_mkr = neutral_RIC.values[0,2::3]
c3d['data']['points'] = np.random.rand(4, len(neutral_lbls), neutral_RIC.shape[0])
c3d['data']['points'][0, :, :] = x_mkr[np.newaxis,:].T
c3d['data']['points'][1, :, :] = y_mkr[np.newaxis,:].T
c3d['data']['points'][2, :, :] = z_mkr[np.newaxis,:].T
c3d['data']['points'][3, :, :] = np.ones((len(z_mkr),1))

# Write the data
#c3d.write(os.path.join(r'../data','20140515T133244_static.c3d'))

Export running to c3d

In [ ]:
# Load an empty c3d structure
c3d_R = c3d()

gait_lbls = list(dict.fromkeys([mkr[:-2] for mkr in gait_RIC.columns.tolist()]))
# Fill it with random data
c3d_R['parameters']['POINT']['RATE']['value'] = [hz_RIC]
c3d_R['parameters']['POINT']['LABELS']['value'] = gait_lbls

x_mkr = gait_RIC.values[:,0:-2:3]
y_mkr = gait_RIC.values[:,1:-1:3]
z_mkr = gait_RIC.values[:,2::3]
c3d_R['data']['points'] = np.random.rand(4, len(gait_lbls), gait_RIC.shape[0])
c3d_R['data']['points'][0, :, :] = x_mkr.T
c3d_R['data']['points'][1, :, :] = y_mkr.T
c3d_R['data']['points'][2, :, :] = z_mkr.T
c3d_R['data']['points'][3, :, :] = np.ones(z_mkr.T.shape)

# Write the data
#c3d_R.write(os.path.join(r'../data','20140515T133244_run.c3d'))

## RBDS

In [ ]:
data_dir = r'C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare'
fn_static = os.path.join(data_dir,'RBDS001static.c3d')
fn_gait = os.path.join(data_dir,'RBDS001runT35.c3d')

In [ ]:
neutral_RBDS, joints_RBDS, gait_RBDS, hz_RBDS = parse_gait.parse_RBDS(fn_static, fn_gait)

Export static to c3d

In [ ]:
# Load an empty c3d structure
c3d = c3d()

neutral_lbls = list(dict.fromkeys([mkr[:-2] for mkr in neutral_RBDS.columns.tolist()]))
# Fill it with random data
c3d['parameters']['POINT']['RATE']['value'] = [hz_RBDS]
c3d['parameters']['POINT']['LABELS']['value'] = neutral_lbls

x_mkr = neutral_RBDS.values[0,0:-2:3]
y_mkr = neutral_RBDS.values[0,1:-1:3]
z_mkr = neutral_RBDS.values[0,2::3]
c3d['data']['points'] = np.random.rand(4, len(neutral_lbls), neutral_RBDS.shape[0])
c3d['data']['points'][0, :, :] = x_mkr[np.newaxis,:].T
c3d['data']['points'][1, :, :] = y_mkr[np.newaxis,:].T
c3d['data']['points'][2, :, :] = z_mkr[np.newaxis,:].T
c3d['data']['points'][3, :, :] = np.ones((len(z_mkr),1))

# Write the data
#c3d.write(os.path.join(r'../data','RBDS001static_rot.c3d'))

Export running to c3d

In [ ]:
# Load an empty c3d structure
c3d_R = c3d()

gait_lbls = list(dict.fromkeys([mkr[:-2] for mkr in gait_RBDS.columns.tolist()]))
# Fill it with random data
c3d_R['parameters']['POINT']['RATE']['value'] = [hz_RBDS]
c3d_R['parameters']['POINT']['LABELS']['value'] = gait_lbls

x_mkr = gait_RBDS.values[:,0:-2:3]
y_mkr = gait_RBDS.values[:,1:-1:3]
z_mkr = gait_RBDS.values[:,2::3]
c3d_R['data']['points'] = np.random.rand(4, len(gait_lbls), gait_RBDS.shape[0])
c3d_R['data']['points'][0, :, :] = x_mkr.T
c3d_R['data']['points'][1, :, :] = y_mkr.T
c3d_R['data']['points'][2, :, :] = z_mkr.T
c3d_R['data']['points'][3, :, :] = np.ones(z_mkr.T.shape)

# Write the data
#c3d.write(os.path.join(r'../data','RBDS001runT35_rot.c3d'))

### Supporting functions

In [ ]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    import scipy.io as spio
    
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def loadjson(filename):
    import json # import library
    with open(fn_json, 'r') as f:
        data = json.load(f)
    
    return data

def cardanangles(r):
    '''
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # %   This function inputs the ROTATION matrix for a given joint
    # %   at one point in time and uses the cardan angles sequence to
    # %   provide the XYZ joint angles in OUT argument
    # %
    # %   The function uses the following  rotation matrix to calculate angles
    # %      | CzCy-SzSySx  SzCy+CzSySx  -SyCx |
    # %      | -SzCx        CzCx         Sx    |
    # %      | CzSy+SzCySx  SzSy-CzCySx  CyCx  |
    # %  INPUTS
    # %  --------
    # %   R (mat):    A 3x3 rotation matrix for a joint

    # %  OUTPUTS
    # %  -------
    # %  OUT (mat):      The three (1x3) planes of rotation in radians
    # %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # %%
    # %   angle.L_ankle(i,:) = cardanangles(R.L_ankle(:,:,i));

    # % the use of atan2 increases the stability by avoiding gimble lock in
    # % the physiologicaly posible range of joint angles.
    '''
    # Prepare Python environment
    import numpy as np
    x = np.arctan2(r[1,2], np.sqrt(r[0,2]**2+r[2,2]**2))
    y = np.arctan2(-r[0,2], r[2,2])
    z = np.arctan2(-r[1,0], r[1,1])

    out = np.array([x,y,z])

    return out

### Debugg angles
Try to find the source of difference in the outputs of RIC and RBDS angles. So far the only direction that seems consistent is the Z-axis.

Start by checking the foot markers and reference frame.

RBDS Lab Coordinate system
*       X - points to the driection of walking
*       Y - points vertically upwards
*       Z - points to the subject's right

RIC the following Lab Coordinate system
*       X - points to the subject's right
*       Y - points vertically upwards
*       Z - points opposite of the walking direction

%% NOTE the Segment coordinate systems
*       X - Anterior .....................................[AB / AD duction]
*       Y - Vertically upwards ............................[Axial rotation]
*       Z - points to the subject's right side ...[Hinge flexion extension]

In [ ]:
R_foot_RBDS = np.array([1359.63320882,   90.7437733 , -638.93188558, 1360.09779948,
          42.57510274, -627.51655965, 1403.29437826,   58.93910467,
        -658.38056478, 1374.34179552,   64.08599357, -641.60967]).reshape((4,3))
print('RBDS foot markers')
print(R_foot_RBDS)

In [ ]:
#%% RIGHT FOOT
#% long axis of the the foot is aligned with the lab
r_foot_x = np.array([0, 0, -1])

#% vertical axis of the foot is aligned with the two markers on the heel
#% FIRST need to identify the heel markers
#% Combine 3 feet markers into one matrix ... can ignore the created fourth one
R_foot = R_foot_RBDS[:-1,:]

# sort the markers from left to right
R_foot = R_foot[R_foot[:, 0].argsort()]

# SECOND, create a vector from the two left markers (not the lateral one)
r_foot_temp = (R_foot[0,:]-R_foot[1,:])/np.linalg.norm(R_foot[0,:]-R_foot[1,:])

#% check that it is pointing up ... and if not, flip it
if r_foot_temp[1] < 0:
    r_foot_temp = -r_foot_temp
    
#% use the temp vertical axis to create the lateral axis
r_foot_z = np.cross(r_foot_x,r_foot_temp)/np.linalg.norm(np.cross(r_foot_x,
                                                                  r_foot_temp))

#% and create the 'vertical' axis that provides standing eversion angle
r_foot_y = np.cross(r_foot_z,r_foot_x)/np.linalg.norm(np.cross(r_foot_z, 
                                                               r_foot_x))

#% combine to create a transformation matrix from anatomical to global
agR_foot = np.array([r_foot_x,r_foot_y,r_foot_z]).T
print('Versors of the foot segment from anatomical to global')
print(agR_foot)

In [ ]:
[[ 0.          0.07124705  0.9974587 ]
 [ 0.          0.9974587  -0.07124705]
 [-1.          0.          0.        ]]

In [ ]:
R_foot_RIC = np.array([342.6       ,  48.1       , -38.2       , 344.9       ,
         80.3       , -46.        , 378.4       ,  39.6       ,
        -65.9       , 355.3       ,  56.        , -50.06666667]).reshape((4,3))
print('RIC foot markers')
print(R_foot_RIC)

In [ ]:
#%% RIGHT FOOT
#% long axis of the the foot is aligned with the lab
r_foot_x = np.array([0, 0, -1])

#% vertical axis of the foot is aligned with the two markers on the heel
#% FIRST need to identify the heel markers
#% Combine 3 feet markers into one matrix ... can ignore the created fourth one
R_foot = R_foot_RIC[:-1,:]

# sort the markers from left to right
R_foot = R_foot[R_foot[:, 0].argsort()]

# SECOND, create a vector from the two left markers (not the lateral one)
r_foot_temp = (R_foot[0,:]-R_foot[1,:])/np.linalg.norm(R_foot[0,:]-R_foot[1,:])

#% check that it is pointing up ... and if not, flip it
if r_foot_temp[1] < 0:
    r_foot_temp = -r_foot_temp
    
#% use the temp vertical axis to create the lateral axis
r_foot_z = np.cross(r_foot_x,r_foot_temp)/np.linalg.norm(np.cross(r_foot_x,
                                                                  r_foot_temp))

#% and create the 'vertical' axis that provides standing eversion angle
r_foot_y = np.cross(r_foot_z,r_foot_x)/np.linalg.norm(np.cross(r_foot_z, 
                                                               r_foot_x))

#% combine to create a transformation matrix from anatomical to global
agR_foot = np.array([r_foot_x,r_foot_y,r_foot_z]).T
print('Versors of the foot segment from anatomical to global')
print(agR_foot)

#### Reproduce method to calculate 3D angles
Provide same output as gait_kinematics.m Matlab function written by Sean Osis.

In [ ]:
# JOINT CENTRES
# Thigh markers
R_lat_knee = joints[['R_lat_knee_X','R_lat_knee_Y','R_lat_knee_Z']].values.flatten()
R_med_knee = joints[['R_med_knee_X','R_med_knee_Y','R_med_knee_Z']].values.flatten()
L_lat_knee = joints[['L_lat_knee_X','L_lat_knee_Y','L_lat_knee_Z']].values.flatten()
L_med_knee = joints[['L_med_knee_X','L_med_knee_Y','L_med_knee_Z']].values.flatten()
L_hip = joints[['L_hip_X','L_hip_Y','L_hip_Z']].values.flatten()
R_hip = joints[['R_hip_X','R_hip_Y','R_hip_Z']].values.flatten()
# Shank markers
R_lat_ankle = joints[['R_lat_ankle_X','R_lat_ankle_Y','R_lat_ankle_Z']].values.flatten()
R_med_ankle = joints[['R_med_ankle_X','R_med_ankle_Y','R_med_ankle_Z']].values.flatten()
L_lat_ankle = joints[['L_lat_ankle_X','L_lat_ankle_Y','L_lat_ankle_Z']].values.flatten()
L_med_ankle = joints[['L_med_ankle_X','L_med_ankle_Y','L_med_ankle_Z']].values.flatten()
# Pelvis markers
pelvis_1 = neutral[['pelvis_1_X','pelvis_1_Y','pelvis_1_Z']].values.flatten()
pelvis_2 = neutral[['pelvis_2_X','pelvis_2_Y','pelvis_2_Z']].values.flatten()
pelvis_3 = neutral[['pelvis_3_X','pelvis_3_Y','pelvis_3_Z']].values.flatten()
pelvis_4 = neutral[['pelvis_4_X','pelvis_4_Y','pelvis_4_Z']].values.flatten()

# the pelvis_jc is simply the average location of the pelvis markers
jcPelvis = (pelvis_1+pelvis_2+pelvis_3+pelvis_4)/4

# HJC
jcL_hip = L_hip + (R_hip-L_hip)/4
jcR_hip = R_hip + (L_hip-R_hip)/4
# midpoint of the two knee markers
jcL_knee = (L_lat_knee+L_med_knee)/2
jcR_knee = (R_lat_knee+R_med_knee)/2
# midpoint of the two ankle markers
jcL_ankle = (L_lat_ankle+L_med_ankle)/2
jcR_ankle = (R_med_ankle+R_lat_ankle)/2

In [ ]:
## LEFT FOOT
#% long axis of the the foot is aligned with the lab
l_foot_x = np.array([0, 0, -1])

#% vertical axis of the foot is aligned with the two markers on the heel
#% FIRST need to identify the heel markers
#% Combine 3 feet markers into one matrix ... can ignore the created fourth one
L_foot = neutral[['L_foot_1_X','L_foot_1_Y','L_foot_1_Z',
                 'L_foot_2_X','L_foot_2_Y','L_foot_2_Z',
                 'L_foot_3_X','L_foot_3_Y','L_foot_3_Z']].values.reshape((3,3))

# sort the markers from left to right
L_foot = L_foot[L_foot[:, 0].argsort()]
# SECOND, create a vector from the two left markers (not the lateral one)
l_foot_temp = (L_foot[1,:]-L_foot[2,:])/np.linalg.norm(L_foot[1,:]-L_foot[2,:])

#% check that it is pointing up ... and if not, flip it
if l_foot_temp[1] < 0:
    l_foot_temp = -l_foot_temp
    
#% use the temp vertical axis to create the lateral axis
l_foot_z = np.cross(l_foot_x,l_foot_temp)/np.linalg.norm(np.cross(l_foot_x,
                                                                  l_foot_temp))

#% and create the 'vertical' axis that provides standing eversion angle
l_foot_y = np.cross(l_foot_z,l_foot_x)/np.linalg.norm(np.cross(l_foot_z, 
                                                               l_foot_x))

#% combine to create a transformation matrix from anatomical to global
agL_foot = np.array([l_foot_x,l_foot_y,l_foot_z]).T
print('Versors of the Left foot segment from anatomical to global')
print(agL_foot)

In [ ]:
#%% RIGHT FOOT
#% long axis of the the foot is aligned with the lab
r_foot_x = np.array([0, 0, -1])

#% vertical axis of the foot is aligned with the two markers on the heel
#% FIRST need to identify the heel markers
#% Combine 3 feet markers into one matrix ... can ignore the created fourth one
R_foot = neutral[['R_foot_1_X','R_foot_1_Y','R_foot_1_Z',
                 'R_foot_2_X','R_foot_2_Y','R_foot_2_Z',
                 'R_foot_3_X','R_foot_3_Y','R_foot_3_Z']].values.reshape((3,3))

# sort the markers from left to right
R_foot = R_foot[R_foot[:, 0].argsort()]
# SECOND, create a vector from the two left markers (not the lateral one)
r_foot_temp = (R_foot[0,:]-R_foot[1,:])/np.linalg.norm(R_foot[0,:]-R_foot[1,:])

#% check that it is pointing up ... and if not, flip it
if r_foot_temp[1] < 0:
    r_foot_temp = -r_foot_temp
    
#% use the temp vertical axis to create the lateral axis
r_foot_z = np.cross(r_foot_x,r_foot_temp)/np.linalg.norm(np.cross(r_foot_x,
                                                                  r_foot_temp))

#% and create the 'vertical' axis that provides standing eversion angle
r_foot_y = np.cross(r_foot_z,r_foot_x)/np.linalg.norm(np.cross(r_foot_z, 
                                                               r_foot_x))

#% combine to create a transformation matrix from anatomical to global
agR_foot = np.array([r_foot_x,r_foot_y,r_foot_z]).T
print('Versors of the foot segment from anatomical to global')
print(agR_foot)

In [ ]:
#%% LEFT SHANK
l_shank_y=(jcL_knee-jcL_ankle)/np.linalg.norm(jcL_knee-jcL_ankle); #long axis pointing up
l_shank_temp=(L_med_ankle-L_lat_ankle)/np.linalg.norm(L_med_ankle-L_lat_ankle) #almost the hinge joing pointing to the right
l_shank_x= np.cross(l_shank_y,l_shank_temp)/np.linalg.norm(np.cross(l_shank_y,l_shank_temp)) #Anterior axis from the cross
l_shank_z=np.cross(l_shank_x,l_shank_y)/np.linalg.norm(np.cross(l_shank_x,l_shank_y)) #hinge axis, lateral for right

# combine to create a transformation matrix from anatomical to global
agL_shank=np.array([l_shank_x,l_shank_y,l_shank_z]).T
print('Versors of the shank segment from anatomical to global')
print(agL_shank)

In [ ]:
#%% RIGHT SHANK
r_shank_y=(jcR_knee-jcR_ankle)/np.linalg.norm(jcR_knee-jcR_ankle); #long axis pointing up
r_shank_temp=(R_lat_ankle-R_med_ankle)/np.linalg.norm(R_lat_ankle-R_med_ankle) #almost the hinge joing pointing to the right
r_shank_x= np.cross(r_shank_y,r_shank_temp)/np.linalg.norm(np.cross(r_shank_y,r_shank_temp)) #Anterior axis from the cross
r_shank_z=np.cross(r_shank_x,r_shank_y)/np.linalg.norm(np.cross(r_shank_x,r_shank_y)) #hinge axis, lateral for right

# combine to create a transformation matrix from anatomical to global
agR_shank=np.array([r_shank_x,r_shank_y,r_shank_z]).T
print('Versors of the shank segment from anatomical to global')
print(agR_shank)

In [ ]:
# LEFT THIGH
l_thigh_y=(jcL_hip-jcL_knee)/np.linalg.norm(jcL_hip-jcL_knee)#long axis pointing up
#almost the hinge joing pointing to the right from the knee joint markers
l_thigh_temp=(L_med_knee-L_lat_knee)/np.linalg.norm(L_med_knee-L_lat_knee)
l_thigh_x=np.cross(l_thigh_y,l_thigh_temp)/np.linalg.norm(np.cross(l_thigh_y,l_thigh_temp))
l_thigh_z=np.cross(l_thigh_x,l_thigh_y)/np.linalg.norm(np.cross(l_thigh_x,l_thigh_y))

# combine to create a transformation matrix from anatomical to global
agL_thigh=np.array([l_thigh_x,l_thigh_y,l_thigh_z]).T
print('Versors of the thigh segment from anatomical to global')
print(agL_thigh)

In [ ]:
# RIGHT THIGH
r_thigh_y=(jcR_hip-jcR_knee)/np.linalg.norm(jcR_hip-jcR_knee)#long axis pointing up
#almost the hinge joing pointing to the right from the knee joint markers
r_thigh_temp=(R_lat_knee-R_med_knee)/np.linalg.norm(R_lat_knee-R_med_knee)
r_thigh_x=np.cross(r_thigh_y,r_thigh_temp)/np.linalg.norm(np.cross(r_thigh_y,r_thigh_temp))
r_thigh_z=np.cross(r_thigh_x,r_thigh_y)/np.linalg.norm(np.cross(r_thigh_x,r_thigh_y))

# combine to create a transformation matrix from anatomical to global
agR_thigh=np.array([r_thigh_x,r_thigh_y,r_thigh_z]).T
print('Versors of the thigh segment from anatomical to global')
print(agR_thigh)

In [ ]:
## PELVIS
# since trochanters are hard to landmark, pelvis will just be orthogonal to the lab
pelvis_x=np.array([0,0,-1])#anterior axis
pelvis_y=np.array([0,1,0])#long axis pointing up
pelvis_z=np.array([1,0,0])#hinge axis to the subject's right
# combine to create a transformation matrix from anatomical to global
ag_Pelvis=np.array([pelvis_x,pelvis_y,pelvis_z]).T

print('Versors of the pelvis segment from anatomical to global')
print(ag_Pelvis)

In [ ]:
# %% use the transformation matrixes we just made to get the segment markers
# % in terms of the ANATOMICAL COORDINATE system (ie if the anatomical
# % coordinate system was located at the world orgin, where would the markers
# % be located). this can be better understood by visualizing the vectors
# % defining the marker location.
# % the marker locations in this reference frame will be used with the
# % soderkvist method as 'before' inputs.

# Rename foot markers in the neutral standing trial
L_foot_1_S = neutral[['L_foot_1_X','L_foot_1_Y','L_foot_1_Z']].values
L_foot_2_S = neutral[['L_foot_2_X','L_foot_2_Y','L_foot_2_Z']].values
L_foot_3_S = neutral[['L_foot_3_X','L_foot_3_Y','L_foot_3_Z']].values
L_foot_4_S = neutral[['L_foot_4_X','L_foot_4_Y','L_foot_4_Z']].values

# % the foot markers in an ANATOMICAL COORDINATE system ...
acL_foot_1=np.dot(agL_foot.T, L_foot_1_S.T)
acL_foot_2=np.dot(agL_foot.T, L_foot_2_S.T)
acL_foot_3=np.dot(agL_foot.T, L_foot_3_S.T)
acL_foot_4=np.dot(agL_foot.T, L_foot_4_S.T)
acL_foot=np.array([acL_foot_1, acL_foot_2, acL_foot_3, acL_foot_4]).T[0,:,:]

# Centroid of markers
centroid = np.mean(acL_foot, axis=1)
# Ankle joint centre in the Foot CS
djcL_ankle=np.dot(agL_foot.T, jcL_ankle.T) - centroid

print('foot markers in an ANATOMICAL COORDINATE system')
print(acL_foot)
print('Ankle joint centre in the Foot CS')
print(djcL_ankle)

In [ ]:
# Rename foot markers in the neutral standing trial
R_foot_1_S = neutral[['R_foot_1_X','R_foot_1_Y','R_foot_1_Z']].values
R_foot_2_S = neutral[['R_foot_2_X','R_foot_2_Y','R_foot_2_Z']].values
R_foot_3_S = neutral[['R_foot_3_X','R_foot_3_Y','R_foot_3_Z']].values
R_foot_4_S = neutral[['R_foot_4_X','R_foot_4_Y','R_foot_4_Z']].values

# % the foot markers in an ANATOMICAL COORDINATE system ...
acR_foot_1=np.dot(agR_foot.T, R_foot_1_S.T)
acR_foot_2=np.dot(agR_foot.T, R_foot_2_S.T)
acR_foot_3=np.dot(agR_foot.T, R_foot_3_S.T)
acR_foot_4=np.dot(agR_foot.T, R_foot_4_S.T)
acR_foot=np.array([acR_foot_1, acR_foot_2, acR_foot_3, acR_foot_4]).T[0,:,:]

# Centroid of markers
centroid = np.mean(acR_foot, axis=1)
# Ankle joint centre in the Foot CS
djcR_ankle=np.dot(agR_foot.T, jcR_ankle.T) - centroid

print('foot markers in an ANATOMICAL COORDINATE system')
print(acR_foot)
print('Ankle joint centre in the Foot CS')
print(djcR_ankle)

In [ ]:
# Rename LEFT shank markers in the neutral standing trial
L_shank_1_S = neutral[['L_shank_1_X','L_shank_1_Y','L_shank_1_Z']].values
L_shank_2_S = neutral[['L_shank_2_X','L_shank_2_Y','L_shank_2_Z']].values
L_shank_3_S = neutral[['L_shank_3_X','L_shank_3_Y','L_shank_3_Z']].values
L_shank_4_S = neutral[['L_shank_4_X','L_shank_4_Y','L_shank_4_Z']].values

# % the shank markers in an ANATOMICAL COORDINATE system ...
acL_shank_1=np.dot(agL_shank.T, L_shank_1_S.T)
acL_shank_2=np.dot(agL_shank.T, L_shank_2_S.T)
acL_shank_3=np.dot(agL_shank.T, L_shank_3_S.T)
acL_shank_4=np.dot(agL_shank.T, L_shank_4_S.T)
acL_shank=np.array([acL_shank_1, acL_shank_2, acL_shank_3, acL_shank_4]).T[0,:,:]
# Centroid of markers
centroid = np.mean(acL_shank, axis=1)
# Knee joint centre in the Foot CS
djcL_knee=np.dot(agL_shank.T, jcL_knee.T) - centroid

print('shank markers in an ANATOMICAL COORDINATE system')
print(acL_shank)
print('knee joint centre in the Foot CS')
print(djcL_knee)

In [ ]:
# Rename RIGHT shank markers in the neutral standing trial
R_shank_1_S = neutral[['R_shank_1_X','R_shank_1_Y','R_shank_1_Z']].values
R_shank_2_S = neutral[['R_shank_2_X','R_shank_2_Y','R_shank_2_Z']].values
R_shank_3_S = neutral[['R_shank_3_X','R_shank_3_Y','R_shank_3_Z']].values
R_shank_4_S = neutral[['R_shank_4_X','R_shank_4_Y','R_shank_4_Z']].values

# % the shank markers in an ANATOMICAL COORDINATE system ...
acR_shank_1=np.dot(agR_shank.T, R_shank_1_S.T)
acR_shank_2=np.dot(agR_shank.T, R_shank_2_S.T)
acR_shank_3=np.dot(agR_shank.T, R_shank_3_S.T)
acR_shank_4=np.dot(agR_shank.T, R_shank_4_S.T)
acR_shank=np.array([acR_shank_1, acR_shank_2, acR_shank_3, acR_shank_4]).T[0,:,:]
# Centroid of markers
centroid = np.mean(acR_shank, axis=1)
# Knee joint centre in the Foot CS
djcR_knee=np.dot(agR_shank.T, jcR_knee.T) - centroid

print('shank markers in an ANATOMICAL COORDINATE system')
print(acR_shank)
print('knee joint centre in the Foot CS')
print(djcR_knee)

In [ ]:
# Rename LEFT thigh markers in the neutral standing trial
L_thigh_1_S = neutral[['L_thigh_1_X','L_thigh_1_Y','L_thigh_1_Z']].values
L_thigh_2_S = neutral[['L_thigh_2_X','L_thigh_2_Y','L_thigh_2_Z']].values
L_thigh_3_S = neutral[['L_thigh_3_X','L_thigh_3_Y','L_thigh_3_Z']].values
L_thigh_4_S = neutral[['L_thigh_4_X','L_thigh_4_Y','L_thigh_4_Z']].values

# % the thigh markers in an ANATOMICAL COORDINATE system ...
acL_thigh_1=np.dot(agL_thigh.T, L_thigh_1_S.T)
acL_thigh_2=np.dot(agL_thigh.T, L_thigh_2_S.T)
acL_thigh_3=np.dot(agL_thigh.T, L_thigh_3_S.T)
acL_thigh_4=np.dot(agL_thigh.T, L_thigh_4_S.T)
acL_thigh=np.array([acL_thigh_1, acL_thigh_2, acL_thigh_3, acL_thigh_4]).T[0,:,:]

# Centroid of markers
centroid = np.mean(acL_thigh, axis=1)
# Knee joint centre in the Foot CS
djcL_hip=np.dot(agL_thigh.T, jcL_hip.T) - centroid

print('thigh markers in an ANATOMICAL COORDINATE system')
print(acL_thigh)
print('hip joint centre in the Foot CS')
print(djcL_hip)

In [ ]:
# Rename thigh markers in the neutral standing trial
R_thigh_1_S = neutral[['R_thigh_1_X','R_thigh_1_Y','R_thigh_1_Z']].values
R_thigh_2_S = neutral[['R_thigh_2_X','R_thigh_2_Y','R_thigh_2_Z']].values
R_thigh_3_S = neutral[['R_thigh_3_X','R_thigh_3_Y','R_thigh_3_Z']].values
R_thigh_4_S = neutral[['R_thigh_4_X','R_thigh_4_Y','R_thigh_4_Z']].values

# % the thigh markers in an ANATOMICAL COORDINATE system ...
acR_thigh_1=np.dot(agR_thigh.T, R_thigh_1_S.T)
acR_thigh_2=np.dot(agR_thigh.T, R_thigh_2_S.T)
acR_thigh_3=np.dot(agR_thigh.T, R_thigh_3_S.T)
acR_thigh_4=np.dot(agR_thigh.T, R_thigh_4_S.T)
acR_thigh=np.array([acR_thigh_1, acR_thigh_2, acR_thigh_3, acR_thigh_4]).T[0,:,:]

# Centroid of markers
centroid = np.mean(acR_thigh, axis=1)
# Knee joint centre in the Foot CS
djcR_hip=np.dot(agR_thigh.T, jcR_hip.T) - centroid

print('thigh markers in an ANATOMICAL COORDINATE system')
print(acR_thigh)
print('hip joint centre in the Foot CS')
print(djcR_hip)

In [ ]:
# Rename Pelvic markers in the neutral standing trial
pelvis_1_S = neutral[['pelvis_1_X','pelvis_1_Y','pelvis_1_Z']].values
pelvis_2_S = neutral[['pelvis_2_X','pelvis_2_Y','pelvis_2_Z']].values
pelvis_3_S = neutral[['pelvis_3_X','pelvis_3_Y','pelvis_3_Z']].values
pelvis_4_S = neutral[['pelvis_4_X','pelvis_4_Y','pelvis_4_Z']].values

# % the pelvis markers in an ANATOMICAL COORDINATE system ...
ac_Pelvis_1=np.dot(ag_Pelvis.T, pelvis_1_S.T)
ac_Pelvis_2=np.dot(ag_Pelvis.T, pelvis_2_S.T)
ac_Pelvis_3=np.dot(ag_Pelvis.T, pelvis_3_S.T)
ac_Pelvis_4=np.dot(ag_Pelvis.T, pelvis_4_S.T)
ac_Pelvis=np.array([ac_Pelvis_1, ac_Pelvis_2, ac_Pelvis_3, ac_Pelvis_4]).T[0,:,:]

# Centroid of markers
centroid = np.mean(ac_Pelvis, axis=1)
# Pelvis joint centre in the Foot CS
djcPelvis=np.dot(ag_Pelvis.T, jcPelvis.T) - centroid

print('Pelvic markers in an ANATOMICAL COORDINATE system')
print(ac_Pelvis)
print('Pelvic centroid in the Pelvic CS')
print(djcPelvis)

In [ ]:
# %% create the rotation matrix from segment to lab for all the segments for
# % every time point. this uses soderkvist. the operations done on the
# % neutral trial data only need to be done once and can be done before we
# % load the dynamic data and are therefore outside of the loop

# % calculate average marker position of the segment markers
avg_ac_L_foot =np.mean(acL_foot,axis=1)
avg_ac_R_foot =np.mean(acR_foot,axis=1)
avg_ac_L_shank=np.mean(acL_shank,axis=1)
avg_ac_R_shank=np.mean(acR_shank,axis=1)
avg_ac_L_thigh=np.mean(acL_thigh,axis=1)
avg_ac_R_thigh=np.mean(acR_thigh,axis=1)
avg_ac_Pelvis =np.mean(ac_Pelvis,axis=1)

# % calculate the distance from the marker posisiton to the average position
# % neutral
dif_ac_L_foot=np.array([acL_foot[0,:]-avg_ac_L_foot[0],
                       acL_foot[1,:]-avg_ac_L_foot[1],
                       acL_foot[2,:]-avg_ac_L_foot[2]])
dif_ac_R_foot=np.array([acR_foot[0,:]-avg_ac_R_foot[0],
                       acR_foot[1,:]-avg_ac_R_foot[1],
                       acR_foot[2,:]-avg_ac_R_foot[2]])
dif_ac_L_shank=np.array([acL_shank[0,:]-avg_ac_L_shank[0],
                       acL_shank[1,:]-avg_ac_L_shank[1],
                       acL_shank[2,:]-avg_ac_L_shank[2]])
dif_ac_R_shank=np.array([acR_shank[0,:]-avg_ac_R_shank[0],
                       acR_shank[1,:]-avg_ac_R_shank[1],
                       acR_shank[2,:]-avg_ac_R_shank[2]])
dif_ac_L_thigh=np.array([acL_thigh[0,:]-avg_ac_L_thigh[0],
                       acL_thigh[1,:]-avg_ac_L_thigh[1],
                       acL_thigh[2,:]-avg_ac_L_thigh[2]])
dif_ac_R_thigh=np.array([acR_thigh[0,:]-avg_ac_R_thigh[0],
                       acR_thigh[1,:]-avg_ac_R_thigh[1],
                       acR_thigh[2,:]-avg_ac_R_thigh[2]])
dif_ac_Pelvis=np.array([ac_Pelvis[0,:]-avg_ac_Pelvis[0],
                       ac_Pelvis[1,:]-avg_ac_Pelvis[1],
                       ac_Pelvis[2,:]-avg_ac_Pelvis[2]])

In [ ]:
# %% now we need the DYNAMIC walking/running data from the input argument
# Left Foot marker positions running
L_foot_1_R = gait.filter(like='L_foot_1', axis=1).values
L_foot_2_R = gait.filter(like='L_foot_2', axis=1).values
L_foot_3_R = gait.filter(like='L_foot_3', axis=1).values
L_foot_4_R = gait.filter(like='L_foot_4', axis=1).values
# Right Foot marker positions running
R_foot_1_R = gait.filter(like='R_foot_1', axis=1).values
R_foot_2_R = gait.filter(like='R_foot_2', axis=1).values
R_foot_3_R = gait.filter(like='R_foot_3', axis=1).values
R_foot_4_R = gait.filter(like='R_foot_4', axis=1).values
# left Shank marker positions running
L_shank_1_R = gait.filter(like='L_shank_1', axis=1).values
L_shank_2_R = gait.filter(like='L_shank_2', axis=1).values
L_shank_3_R = gait.filter(like='L_shank_3', axis=1).values
L_shank_4_R = gait.filter(like='L_shank_4', axis=1).values
# Right Shank marker positions running
R_shank_1_R = gait.filter(like='R_shank_1', axis=1).values
R_shank_2_R = gait.filter(like='R_shank_2', axis=1).values
R_shank_3_R = gait.filter(like='R_shank_3', axis=1).values
R_shank_4_R = gait.filter(like='R_shank_4', axis=1).values
# left Thigh marker positions running
L_thigh_1_R = gait.filter(like='L_thigh_1', axis=1).values
L_thigh_2_R = gait.filter(like='L_thigh_2', axis=1).values
L_thigh_3_R = gait.filter(like='L_thigh_3', axis=1).values
L_thigh_4_R = gait.filter(like='L_thigh_4', axis=1).values
# Thigh marker positions running
R_thigh_1_R = gait.filter(like='R_thigh_1', axis=1).values
R_thigh_2_R = gait.filter(like='R_thigh_2', axis=1).values
R_thigh_3_R = gait.filter(like='R_thigh_3', axis=1).values
R_thigh_4_R = gait.filter(like='R_thigh_4', axis=1).values
# Pelvis marker positions running
pelvis_1_R = gait.filter(like='pelvis_1', axis=1).values
pelvis_2_R = gait.filter(like='pelvis_2', axis=1).values
pelvis_3_R = gait.filter(like='pelvis_3', axis=1).values
pelvis_4_R = gait.filter(like='pelvis_4', axis=1).values

# % prealocating the size of these matricies that collect data in the loop
# % to increase speed
# RIGHT SIDE
R_R_foot  =np.zeros(shape=(4,4, R_foot_1_R.shape[0]))
R_R_shank =np.zeros(shape=(4,4, R_foot_1_R.shape[0]))
R_R_thigh =np.zeros(shape=(4,4, R_foot_1_R.shape[0]))
R_R_pelvis=np.zeros(shape=(4,4, R_foot_1_R.shape[0]))
#pre-allaocate angles
R_R_ankle=np.zeros(shape=(4,4, R_foot_1_R.shape[0]))
R_R_knee =np.zeros(shape=(4,4, R_foot_1_R.shape[0]))
R_R_hip  =np.zeros(shape=(4,4, R_foot_1_R.shape[0]))

# LEFT SIDE
R_L_foot  =np.zeros(shape=(4,4, L_foot_1_R.shape[0]))
R_L_shank =np.zeros(shape=(4,4, L_foot_1_R.shape[0]))
R_L_thigh =np.zeros(shape=(4,4, L_foot_1_R.shape[0]))
R_L_pelvis=np.zeros(shape=(4,4, L_foot_1_R.shape[0]))
#pre-allaocate angles
R_L_ankle=np.zeros(shape=(4,4, L_foot_1_R.shape[0]))
R_L_knee =np.zeros(shape=(4,4, L_foot_1_R.shape[0]))
R_L_hip  =np.zeros(shape=(4,4, L_foot_1_R.shape[0]))

In [ ]:
# %% need to create matricies from the DYNAMIC data that has the same markers
# % as the " ac.L_foot " in the same order ... matricies for use with soderqvist
dL_foot = np.array([L_foot_1_R.T, L_foot_2_R.T, L_foot_3_R.T, L_foot_4_R.T], ndmin=3)
dL_foot = np.transpose(dL_foot, (1,0,2))
dR_foot = np.array([R_foot_1_R.T, R_foot_2_R.T, R_foot_3_R.T, R_foot_4_R.T], ndmin=3)
dR_foot = np.transpose(dR_foot, (1,0,2))
dL_shank = np.array([L_shank_1_R.T, L_shank_2_R.T, L_shank_3_R.T, L_shank_4_R.T], ndmin=3)
dL_shank = np.transpose(dL_shank, (1,0,2))
dR_shank = np.array([R_shank_1_R.T, R_shank_2_R.T, R_shank_3_R.T, R_shank_4_R.T], ndmin=3)
dR_shank = np.transpose(dR_shank, (1,0,2))
dL_thigh = np.array([L_thigh_1_R.T, L_thigh_2_R.T, L_thigh_3_R.T, L_thigh_4_R.T], ndmin=3)
dL_thigh = np.transpose(dL_thigh, (1,0,2))
dR_thigh = np.array([R_thigh_1_R.T, R_thigh_2_R.T, R_thigh_3_R.T, R_thigh_4_R.T], ndmin=3)
dR_thigh = np.transpose(dR_thigh, (1,0,2))
dPelvis = np.array([pelvis_1_R.T, pelvis_2_R.T, pelvis_3_R.T, pelvis_4_R.T], ndmin=3)
dPelvis = np.transpose(dPelvis, (1,0,2))

# calculate average position of the segment markers
avg_dL_foot = dL_foot.mean(axis=1)
avg_dL_foot = np.repeat(avg_dL_foot[:, np.newaxis,:], 4, axis=1)
avg_dR_foot = dR_foot.mean(axis=1)
avg_dR_foot = np.repeat(avg_dR_foot[:, np.newaxis,:], 4, axis=1)
avg_dL_shank= dL_shank.mean(axis=1)
avg_dL_shank = np.repeat(avg_dL_shank[:, np.newaxis,:], 4, axis=1)
avg_dR_shank= dR_shank.mean(axis=1)
avg_dR_shank = np.repeat(avg_dR_shank[:, np.newaxis,:], 4, axis=1)
avg_dL_thigh= dL_thigh.mean(axis=1)
avg_dL_thigh = np.repeat(avg_dL_thigh[:, np.newaxis,:], 4, axis=1)
avg_dR_thigh= dR_thigh.mean(axis=1)
avg_dR_thigh = np.repeat(avg_dR_thigh[:, np.newaxis,:], 4, axis=1)
avg_dPelvis= dPelvis.mean(axis=1)
avg_dPelvis = np.repeat(avg_dPelvis[:, np.newaxis,:], 4, axis=1)

# calculate the distance from the marker posisiton to the average position
dif_dL_foot = dL_foot - avg_dL_foot
dif_dR_foot = dR_foot - avg_dR_foot
dif_dL_shank= dL_shank - avg_dL_shank
dif_dR_shank= dR_shank - avg_dR_shank
dif_dL_thigh= dL_thigh - avg_dL_thigh
dif_dR_thigh= dR_thigh - avg_dR_thigh
dif_dPelvis= dPelvis - avg_dPelvis

In [ ]:
# Repeat array to perform dot product on transposed array
# Uncomment this if you figure out how to do without loop using 
# np.einsum('ijk,ij->ij', dif_dR_foot, dif_ac_R_foot)
# dif_ac_R_footT = dif_ac_R_foot.T # Tranpose array
# dif_ac_R_footT = np.repeat(dif_ac_R_footT[:,:,np.newaxis], 
#                           dif_dR_foot.shape[2], axis=2)

# Preallocate angles
angle_L_ankle = np.empty(shape=(dif_dL_foot.shape[2], 3)) * np.NaN
angle_R_ankle = np.empty(shape=(dif_dR_foot.shape[2], 3)) * np.NaN
angle_L_knee  = np.empty(shape=(dif_dL_foot.shape[2], 3)) * np.NaN
angle_R_knee  = np.empty(shape=(dif_dR_foot.shape[2], 3)) * np.NaN
angle_L_hip  = np.empty(shape=(dif_dL_foot.shape[2], 3)) * np.NaN
angle_R_hip  = np.empty(shape=(dif_dR_foot.shape[2], 3)) * np.NaN
angle_L_foot = np.empty(shape=(dif_dL_foot.shape[2], 3)) * np.NaN
angle_R_foot = np.empty(shape=(dif_dR_foot.shape[2], 3)) * np.NaN
angle_Pelvis = np.empty(shape=(dif_dR_foot.shape[2], 3)) * np.NaN
for i in range(dif_dR_foot.shape[2]):
    # step three of soderkvist
    CL_foot = np.dot(dif_dL_foot[:,:,i], dif_ac_L_foot.T)
    CR_foot = np.dot(dif_dR_foot[:,:,i], dif_ac_R_foot.T)
    CL_shank= np.dot(dif_dL_shank[:,:,i], dif_ac_L_shank.T)
    CR_shank= np.dot(dif_dR_shank[:,:,i], dif_ac_R_shank.T)
    CL_thigh= np.dot(dif_dL_thigh[:,:,i], dif_ac_L_thigh.T)
    CR_thigh= np.dot(dif_dR_thigh[:,:,i], dif_ac_R_thigh.T)
    CPelvis= np.dot(dif_dPelvis[:,:,i], dif_ac_Pelvis.T)
    
    # step four singular value decomposition
    PL_foot, TL_foot, QL_foot = np.linalg.svd(CL_foot)
    TL_foot = np.diag(TL_foot)
    
    PR_foot, TR_foot, QR_foot = np.linalg.svd(CR_foot)
    TR_foot = np.diag(TR_foot)

    PL_shank, TL_shank, QL_shank = np.linalg.svd(CL_shank)
    TL_shank = np.diag(TL_shank)
    
    PR_shank, TR_shank, QR_shank = np.linalg.svd(CR_shank)
    TR_shank = np.diag(TR_shank)

    PL_thigh, TL_thigh, QL_thigh = np.linalg.svd(CL_thigh)
    TL_thigh = np.diag(TL_thigh)
    
    PR_thigh, TR_thigh, QR_thigh = np.linalg.svd(CR_thigh)
    TR_thigh = np.diag(TR_thigh)
    
    PPelvis, TPelvis, QPelvis = np.linalg.svd(CPelvis)
    TPelvis = np.diag(TPelvis)
    
    # step five - calculate a rotation matrix
    # Foot
    R_L_foot = np.dot(PL_foot, np.dot(np.diag([1, 1, 
                                           np.linalg.det(np.dot(PL_foot, 
                                                                QL_foot))]), 
                                  QL_foot))
    
    R_R_foot = np.dot(PR_foot, np.dot(np.diag([1, 1, 
                                               np.linalg.det(np.dot(PR_foot, 
                                                                    QR_foot))]), 
                                      QR_foot))
    # Shank
    R_L_shank = np.dot(PL_shank, np.dot(np.diag([1, 1, 
                                           np.linalg.det(np.dot(PL_shank, 
                                                                QL_shank))]), 
                                  QL_shank))
    
    R_R_shank = np.dot(PR_shank, np.dot(np.diag([1, 1, 
                                               np.linalg.det(np.dot(PR_shank, 
                                                                    QR_shank))]), 
                                      QR_shank))
    
    # Thigh
    R_L_thigh = np.dot(PL_thigh, np.dot(np.diag([1, 1, 
                                               np.linalg.det(np.dot(PL_thigh, 
                                                                    QL_thigh))]), 
                                      QL_thigh))
    
    R_R_thigh = np.dot(PR_thigh, np.dot(np.diag([1, 1, 
                                               np.linalg.det(np.dot(PR_thigh, 
                                                                    QR_thigh))]), 
                                      QR_thigh))
    
    # Pelvis
    R_Pelvis = np.dot(PPelvis, np.dot(np.diag([1, 1, 
                                               np.linalg.det(np.dot(PPelvis, 
                                                                    QPelvis))]), 
                                      QPelvis))
    
    # step six - calculate the displacement
    dis_L_foot = avg_dL_foot[:,0,-1] - np.dot(R_L_foot, avg_ac_L_foot)
    dis_R_foot = avg_dR_foot[:,0,-1] - np.dot(R_R_foot, avg_ac_R_foot)
    dis_L_shank= avg_dL_shank[:,0,-1] - np.dot(R_L_shank, avg_ac_L_shank)
    dis_R_shank= avg_dR_shank[:,0,-1] - np.dot(R_R_shank, avg_ac_R_shank)
    dis_L_thigh= avg_dL_thigh[:,0,-1] - np.dot(R_L_thigh, avg_ac_L_thigh)
    dis_R_thigh= avg_dR_thigh[:,0,-1] - np.dot(R_R_thigh, avg_ac_R_thigh)
    dis_Pelvis= avg_dPelvis[:,0,-1] - np.dot(R_Pelvis, avg_ac_Pelvis)
    
    # combine the rotation matricies in a stack
    RR_L_foot = np.vstack((np.hstack((R_L_foot,dis_L_foot[:,np.newaxis])),
                      np.array([0,0,0,1], ndmin=2)))
    RR_R_foot = np.vstack((np.hstack((R_R_foot,dis_R_foot[:,np.newaxis])),
                      np.array([0,0,0,1], ndmin=2)))
    RR_L_shank = np.vstack((np.hstack((R_L_shank,dis_L_shank[:,np.newaxis])),
                          np.array([0,0,0,1], ndmin=2)))
    RR_R_shank = np.vstack((np.hstack((R_R_shank,dis_R_shank[:,np.newaxis])),
                          np.array([0,0,0,1], ndmin=2)))
    RR_L_thigh = np.vstack((np.hstack((R_L_thigh,dis_L_thigh[:,np.newaxis])),
                          np.array([0,0,0,1], ndmin=2)))
    RR_R_thigh = np.vstack((np.hstack((R_R_thigh,dis_R_thigh[:,np.newaxis])),
                          np.array([0,0,0,1], ndmin=2)))
    RR_Pelvis = np.vstack((np.hstack((R_Pelvis,dis_Pelvis[:,np.newaxis])),
                          np.array([0,0,0,1], ndmin=2)))
    
    
    # %% calculate segment angles
    #     % FOOT
    #     % the rotation matrix of the foot to lab (R.L_foot) is simply a
    #     % combination of the XYZ axes of the segment coordinate system unit
    #     % vectors
    #     %   footX_labx     footY_labx     footZ_labx
    #     %   footX_laby     footY_laby     footZ_laby
    #     %   footX_labz     footY_labz     footZ_labz
    #     % so in order to calculate foot progression angle / heel whip ...
    #     % Collect the angle of the long axis of the foot about the
    #     % vertical axis of the foot.
    #     % Also by projecting the foot into the sagital plane, we have
    #     % information to determine heelstrikers or FOREFOOT strikers when
    #     % identifying events

    #     % angle of the vertical axis of the foot projected into the frontal plane
    #     %   from a posterior view, 'vertical vector' in the first quadrant is
    #     %   postitive and second quadrant is negative
    #     %   . ie. inversion is negative and eversion is positive for the LEFT
    #     % note that this is projected into the frontal plane so cross talk is
    #     % present ... dorsi flexion with an abducted foot creates lots of "inversion"
    angle_L_foot[i,0] = np.arctan(RR_L_foot[0,1]/np.sqrt(RR_L_foot[1,1]**2 + RR_L_foot[2,1]**2))
    # angle of the long axis of the foot about the vertical axis
    angle_L_foot[i,1] = np.arctan(-RR_L_foot[0,0]/np.sqrt(RR_L_foot[1,0]**2 + RR_L_foot[2,0]**2))
    # and project the long axis into the sagital plane for ID of FOREFOOT
    angle_L_foot[i,2] = np.arctan2(RR_L_foot[1,0], -R_L_foot[2,0])

    #     % angle of the vertical axis of the foot projected into the frontal plane
    #     %   from a posterior view, 'vertical vector' in the first quadrant is
    #     %   postitive and second quadrant is negative
    #     %   . ie. inversion is positive and eversion is negative for the RIGHT
    
    angle_R_foot[i,0] = np.arctan(RR_R_foot[0,1]/np.sqrt(RR_R_foot[1,1]**2 + RR_R_foot[2,1]**2))
    # angle of the long axis of the foot about the vertical axis
    angle_R_foot[i,1] = np.arctan(RR_R_foot[0,0]/np.sqrt(RR_R_foot[1,0]**2 + RR_R_foot[2,0]**2))
    # and project the long axis into the sagital plane for ID of FOREFOOT
    angle_R_foot[i,2] = np.arctan2(RR_R_foot[1,0], -R_R_foot[2,0])
    
    # % PELVIS
    # % project lateral axis of pelvis
    angle_Pelvis[i,0] = np.arctan2(RR_Pelvis[0,2],RR_Pelvis[2,2]) - np.pi/2 #into floor plane
    angle_Pelvis[i,1] = np.arctan(RR_Pelvis[1,2]/RR_Pelvis[0,2]) #into frontal plane
    #% and project anterior axis of pelvis
    angle_Pelvis[i,2] = np.arctan2(RR_Pelvis[1,0],-RR_Pelvis[2,0])#into sagital plane

    # %% calculate joint angles
    # % need rotation matrix from shank to foot ...
    # % so multiply [lab to shank] with [foot to lab]
    R_L_ankle = np.dot(RR_L_shank.T, RR_L_foot)
    R_R_ankle = np.dot(RR_R_shank.T, RR_R_foot)
    R_L_knee = np.dot(RR_L_thigh.T, RR_L_shank)
    R_R_knee = np.dot(RR_R_thigh.T, RR_R_shank)
    R_L_hip = np.dot(RR_Pelvis.T, RR_L_thigh)
    R_R_hip = np.dot(RR_Pelvis.T, RR_R_thigh)
    
    #np.set_printoptions(suppress=True)


    # % CARDANANGLES uses this rotation matrix to calculate angles
    #     %   | CzCy-SzSySx  SzCy+CzSySx  -SyCx |
    #     %   | -SzCx        CzCx         Sx    |
    #     %   | CzSy+SzCySx  SzSy-CzCySx  CyCx  |
    angle_L_ankle[i,:] = cardanangles(R_L_ankle)
    angle_R_ankle[i,:] = cardanangles(R_R_ankle)
    angle_L_knee[i,:] = cardanangles(R_L_knee)
    angle_R_knee[i,:] = cardanangles(R_R_knee)
    angle_L_hip[i,:] = cardanangles(R_L_hip)
    angle_R_hip[i,:] = cardanangles(R_R_hip)

### RBDS data

In [ ]:
import parse_gait_kinematics as parse_gait
from gait_kinematics import gait_kinematics
from gait_steps import gait_steps

In [ ]:
data_dir = r'C:\Users\Reginaldo\Documents\data\CNPq\RBDS_v2\Figshare'
fn_static = os.path.join(data_dir,'RBDS001static.c3d')
fn_gait = os.path.join(data_dir,'RBDS001runT35.c3d')
neutral_RBDS, joints_RBDS, gait_RBDS, hz_RBDS = parse_gait.parse_RBDS(fn_static, fn_gait)

angle_L_ankle, angle_R_ankle, angle_L_knee, angle_R_knee, angle_L_hip, angle_R_hip, angle_L_foot, angle_R_foot, angle_Pelvis = gait_kinematics(joints_RBDS, neutral_RBDS, gait_RBDS, hz_RBDS)

# Create dataframe column corresponding to the dataset
joints_lbls = ['pelvis','L_foot','R_foot','L_hip','R_hip','L_knee','R_knee',
               'L_ankle','R_ankle']
xyz = list('XYZ')*len(joints_lbls)
joints_lbls = [ele for ele in joints_lbls for i in range(3)]
joints_lbls = [joints_lbls[i]+'_'+xyz[i] for i in range(len(xyz))]

# Convert to degrees
angle_L_ankle, angle_R_ankle = angle_L_ankle*(180/np.pi), angle_R_ankle*(180/np.pi)
angle_L_knee, angle_R_knee   = angle_L_knee*(180/np.pi), angle_R_knee*(180/np.pi)
angle_L_hip, angle_R_hip     = angle_L_hip*(180/np.pi), angle_R_hip*(180/np.pi)
angle_L_foot, angle_R_foot   = angle_L_foot*(180/np.pi), angle_R_foot*(180/np.pi)
angle_Pelvis  = angle_Pelvis * (180/np.pi)
# Create a pandas df with angles
angs = np.hstack([angle_Pelvis, angle_L_foot, angle_R_foot, 
                  angle_L_hip, angle_R_hip, angle_L_knee, angle_R_knee, 
                  angle_L_ankle, angle_R_ankle])
angles_RBDS = pd.DataFrame(data=angs, columns=joints_lbls)

# Get gait events
L_TD_RBDS, L_TO_RBDS, R_TD_RBDS, R_TO_RBDS, eventsflag_RBDS, label_RBDS = gait_steps(neutral_RBDS, gait_RBDS, angles_RBDS, hz_RBDS)

RTD_RBDS, RTO_RBDS = R_TD_RBDS.astype(int).tolist(), R_TO_RBDS.astype(int).tolist()
LTD_RBDS, LTO_RBDS = L_TD_RBDS.astype(int).tolist(), L_TO_RBDS.astype(int).tolist()

### RIC data

In [ ]:
figshare_dir = r'C:\Users\Reginaldo\OneDrive - University of Calgary\data\Figshare_SciData\new_unzip'
fn_json=os.path.join(figshare_dir, '201225', '20140515T133244.json')
neutral_RIC, joints_RIC, gait_RIC, hz_RIC = parse_gait.parse_RIC(fn_json)

In [ ]:
angle_L_ankle, angle_R_ankle, angle_L_knee, angle_R_knee, angle_L_hip, angle_R_hip, angle_L_foot, angle_R_foot, angle_Pelvis = gait_kinematics(joints_RIC, neutral_RIC, gait_RIC, hz_RIC)
# Create dataframe column corresponding to the dataset
joints_lbls = ['pelvis','L_foot','R_foot','L_hip','R_hip','L_knee','R_knee',
               'L_ankle','R_ankle']
xyz = list('XYZ')*len(joints_lbls)
joints_lbls = [ele for ele in joints_lbls for i in range(3)]
joints_lbls = [joints_lbls[i]+'_'+xyz[i] for i in range(len(xyz))]

# Convert to degrees
angle_L_ankle, angle_R_ankle = angle_L_ankle*(180/np.pi), angle_R_ankle*(180/np.pi)
angle_L_knee, angle_R_knee   = angle_L_knee*(180/np.pi), angle_R_knee*(180/np.pi)
angle_L_hip, angle_R_hip     = angle_L_hip*(180/np.pi), angle_R_hip*(180/np.pi)
angle_L_foot, angle_R_foot   = angle_L_foot*(180/np.pi), angle_R_foot*(180/np.pi)
angle_Pelvis  = angle_Pelvis * (180/np.pi)
# Create a pandas df with angles
angs = np.hstack([angle_Pelvis, angle_L_foot, angle_R_foot, 
                  angle_L_hip, angle_R_hip, angle_L_knee, angle_R_knee, 
                  angle_L_ankle, angle_R_ankle])
angles_RIC = pd.DataFrame(data=angs, columns=joints_lbls)

In [ ]:
# Get gait events
L_TD_RIC, L_TO_RIC, R_TD_RIC, R_TO_RIC, eventsflag_RIC, label_RIC = gait_steps(neutral_RIC, gait_RIC, angles_RIC, hz_RIC)

RTD_RIC, RTO_RIC = R_TD_RIC.astype(int).tolist(), R_TO_RIC.astype(int).tolist()
LTD_RIC, LTO_RIC = L_TD_RIC.astype(int).tolist(), L_TO_RIC.astype(int).tolist()

In [ ]:
joint='knee'

In [ ]:
angle_L_RIC = angles_RIC.filter(like='L_'+joint, axis=1).values
angle_R_RIC = angles_RIC.filter(like='R_'+joint, axis=1).values

In [ ]:
angle_L_RBDS = angles_RBDS.filter(like='L_'+joint, axis=1).values
angle_R_RBDS = angles_RBDS.filter(like='R_'+joint, axis=1).values

In [ ]:
# RIC dataset
fig, axs = plt.subplots(3,2, figsize=(10,6))
fig.suptitle('RIGHT '+joint.upper()+' angles')
# RIC
for i in range(len(RTD_RIC)-1):
    axs[0,0].plot(angle_R_RIC[RTD_RIC[i]:RTD_RIC[i+1],0])
    axs[0,0].set_ylabel('X')
    axs[0,0].grid('on')
    axs[0,0].set_title('RIC')
    axs[1,0].plot(angle_R_RIC[RTD_RIC[i]:RTD_RIC[i+1],1])
    axs[1,0].set_ylabel('Y')
    axs[1,0].grid('on')
    axs[2,0].plot(angle_R_RIC[RTD_RIC[i]:RTD_RIC[i+1],2])
    axs[2,0].set_ylabel('Z')
    axs[2,0].grid('on')
    
for i in range(len(RTD_RBDS)-1):
    axs[0,1].plot(angle_R_RBDS[RTD_RBDS[i]:RTD_RBDS[i+1],0])
    axs[0,1].set_ylabel('X')
    axs[0,1].grid('on')
    axs[0,1].set_title('RBDS')
    axs[1,1].plot(angle_R_RBDS[RTD_RBDS[i]:RTD_RBDS[i+1],1])
    axs[1,1].set_ylabel('Y')
    axs[1,1].grid('on')
    axs[2,1].plot(angle_R_RBDS[RTD_RBDS[i]:RTD_RBDS[i+1],2])
    axs[2,1].set_ylabel('Z')
    axs[2,1].grid('on')
plt.show()

In [ ]:
# RIC dataset
fig, axs = plt.subplots(3,2, figsize=(10,6))
fig.suptitle('LEFT '+joint.upper()+' angles')
# RIC
for i in range(len(RTD_RIC)-1):
    axs[0,0].plot(angle_L_RIC[LTD_RIC[i]:LTD_RIC[i+1],0])
    axs[0,0].set_ylabel('X')
    axs[0,0].grid('on')
    axs[0,0].set_title('RIC')
    axs[1,0].plot(angle_L_RIC[LTD_RIC[i]:LTD_RIC[i+1],1])
    axs[1,0].set_ylabel('Y')
    axs[1,0].grid('on')
    axs[2,0].plot(angle_L_RIC[LTD_RIC[i]:LTD_RIC[i+1],2])
    axs[2,0].set_ylabel('Z')
    axs[2,0].grid('on')
    
for i in range(len(RTD_RBDS)-1):
    axs[0,1].plot(angle_L_RBDS[LTD_RBDS[i]:LTD_RBDS[i+1],0])
    axs[0,1].set_ylabel('X')
    axs[0,1].grid('on')
    axs[0,1].set_title('RBDS')
    axs[1,1].plot(angle_L_RBDS[LTD_RBDS[i]:LTD_RBDS[i+1],1])
    axs[1,1].set_ylabel('Y')
    axs[1,1].grid('on')
    axs[2,1].plot(angle_L_RBDS[LTD_RBDS[i]:LTD_RBDS[i+1],2])
    axs[2,1].set_ylabel('Z')
    axs[2,1].grid('on')
plt.show()

### RIC data

In [ ]:
figshare_dir = r'C:\Users\Reginaldo\OneDrive - University of Calgary\data\Figshare_SciData\new_unzip'
fn_json=os.path.join(figshare_dir, '201225', '20140515T133244.json')
neutral_RIC, joints_RIC, gait_RIC, hz_RIC = parse_gait.parse_RIC(fn_json)

In [ ]:
neutral_RIC

In [ ]:
mkr_S = neutral_RIC.values.flatten()
# Marker labels list
mkr_S_lbl = [neutral_RIC.columns.tolist()[i][:-2] for i in range(mkr_S.shape[0])]
mkr_S_lbl_ = f7(mkr_S_lbl)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8, 10))
ax = fig.add_subplot(111, projection='3d',  facecolor='white')
#ax.view_init(15, 20)

ax.scatter(mkr_S[0:-3:3], mkr_S[2:-1:3], mkr_S[1:-2:3], c='r', s=30, depthshade=False)
for m, mkr_lbl in enumerate(mkr_S_lbl_):
    ax.text(mkr_S[3*m], mkr_S[3*m+2], mkr_S[3*m+1], mkr_lbl)

ax.set_xlim3d([np.nanmin(mkr_S[0::3])-.4, np.nanmax(mkr_S[0::3])+.4])
ax.set_zlim3d([np.nanmin(mkr_S[1::3])-.4, np.nanmax(mkr_S[1::3])+.4])
ax.set_ylim3d([np.nanmin(mkr_S[2::3]), np.nanmax(mkr_S[2::3])])
ax.set_xlabel('\n' + 'X [m]', linespacing=2)
ax.set_zlabel('\n' + 'Y [m]', linespacing=2)
ax.set_ylabel('\n' + 'Z [m]', linespacing=2)
#ax.invert_yaxis()
# square plot
ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
eng.workspace['junta'] = joints_dic

In [ ]:
matlab.double(joints_dic)

In [ ]:



L_FFi, L_FBi, L_block_start, block_end = eng.largest_block(matlab.double(list(L_FFi)), 
                                                             matlab.double(list(L_FBi)), nargout=4)

In [ ]:
[angles,velocities,jc,R,djc] = gait_kinematics(joints,neutral,dynamic,hz,plots)

In [ ]:
mkr_S = neutral_RBDS.values.flatten()
# Marker labels list
mkr_S_lbl = [neutral_RBDS.columns.tolist()[i][:-2] for i in range(mkr_S.shape[0])]
mkr_S_lbl_ = f7(mkr_S_lbl)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8, 10))
ax = fig.add_subplot(111, projection='3d',  facecolor='white')
#ax.view_init(15, 20)

ax.scatter(mkr_S[0:-3:3], mkr_S[2:-1:3], mkr_S[1:-2:3], c='r', s=30, depthshade=False)
for m, mkr_lbl in enumerate(mkr_S_lbl_):
    ax.text(mkr_S[3*m], mkr_S[3*m+2], mkr_S[3*m+1], mkr_lbl)

ax.set_xlim3d([np.nanmin(mkr_S[0::3])-.4, np.nanmax(mkr_S[0::3])+.4])
ax.set_zlim3d([np.nanmin(mkr_S[1::3])-.4, np.nanmax(mkr_S[1::3])+.4])
ax.set_ylim3d([np.nanmin(mkr_S[2::3]), np.nanmax(mkr_S[2::3])])
ax.set_xlabel('\n' + 'X [m]', linespacing=2)
ax.set_zlabel('\n' + 'Y [m]', linespacing=2)
ax.set_ylabel('\n' + 'Z [m]', linespacing=2)
#ax.invert_yaxis()
# square plot
ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
# Import gait events calculated in Matlab
fn_f_matlab = os.path.join(pathname, 'events_RBDS.txt')
events = np.loadtxt(fn_f_matlab)

In [ ]:
events[:,2]

In [ ]:
np.array(R_TD)

In [ ]:
events[:,2]-np.array(R_TD)

## Plot compare outputs

In [ ]:
pathname = r'../functions'

In [ ]:
tn = np.linspace(0,angle_R_ankle.shape[0],num=angle_R_ankle.shape[0])

In [ ]:
# Import foot angles calculated in Matlab
fn_f_matlab = os.path.join(pathname, 'L_foot_angle_RBDSmat.txt')
angf_matlab = np.loadtxt(fn_f_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10,6))
fig.suptitle('Comparison of left foot angles between methods')
axs[0].plot(angle_L_foot[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],angf_matlab[0::5,0], 'bo', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_L_foot[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],angf_matlab[0::5,1], 'ro', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_L_foot[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],angf_matlab[0::5,2], 'go', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import foot angles calculated in Matlab
fn_f_matlab = os.path.join(pathname, 'r_foot_angle_RBDSmat.txt')
angf_matlab = np.loadtxt(fn_f_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10,6))
fig.suptitle('Comparison of right foot angles between methods')
axs[0].plot(angle_R_foot[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],angf_matlab[0::5,0], 'bo', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_R_foot[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],angf_matlab[0::5,1], 'ro', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_R_foot[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],angf_matlab[0::5,2], 'go', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import Pelvis angles calculated in Matlab
fn_p_matlab = os.path.join(pathname, 'pelvis_angle_RBDS.txt')
angp_matlab = np.loadtxt(fn_p_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10,6))
fig.suptitle('Comparison of Pelvis angles between methods')
axs[0].plot(angle_Pelvis[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],angp_matlab[0::5,0], 'bo', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_Pelvis[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],angp_matlab[0::5,1], 'ro', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_Pelvis[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],angp_matlab[0::5,2], 'go', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import ankle angles calculated in Matlab
fn_a_matlab = os.path.join(pathname, 'L_ankle_angle_RBDSmat.txt')
anga_matlab = np.loadtxt(fn_a_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(3, figsize=(10,6))
fig.suptitle('Comparison of LEFT ankle angles between methods')
axs[0].plot(angle_L_ankle[:,0], 'b', label='Python')
axs[0].plot(tn[0::3],anga_matlab[0::3,0], 'b*', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_L_ankle[:,1], 'r', label='Python')
axs[1].plot(tn[0::3],anga_matlab[0::3,1], 'r*', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_L_ankle[:,2], 'g', label='Python')
axs[2].plot(tn[0::3],anga_matlab[0::3,2], 'g*', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import ankle angles calculated in Matlab
fn_a_matlab = os.path.join(pathname, 'r_ankle_angle_RBDSmat.txt')
anga_matlab = np.loadtxt(fn_a_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(3, figsize=(10,6))
fig.suptitle('Comparison of RIGHT ankle angles between methods')
axs[0].plot(angle_R_ankle[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],anga_matlab[0::5,0], 'b*', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_R_ankle[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],anga_matlab[0::5,1], 'r*', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_R_ankle[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],anga_matlab[0::5,2], 'g*', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import ankle angles calculated in Matlab
fn_k_matlab = os.path.join(pathname, 'L_knee_angle_RBDSmat.txt')
angk_matlab = np.loadtxt(fn_k_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(3, figsize=(10,6))
fig.suptitle('Comparison of knee angles between methods')
axs[0].plot(angle_L_knee[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],angk_matlab[0::5,0], 'b*', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_L_knee[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],angk_matlab[0::5,1], 'r*', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_L_knee[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],angk_matlab[0::5,2], 'g*', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import ankle angles calculated in Matlab
fn_k_matlab = os.path.join(pathname, 'r_knee_angle_RBDSmat.txt')
angk_matlab = np.loadtxt(fn_k_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(3, figsize=(10,6))
fig.suptitle('Comparison of knee angles between methods')
axs[0].plot(angle_R_knee[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],angk_matlab[0::5,0], 'b*', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_R_knee[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],angk_matlab[0::5,1], 'r*', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_R_knee[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],angk_matlab[0::5,2], 'g*', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import ankle angles calculated in Matlab
fn_h_matlab = os.path.join(pathname, 'L_hip_angle_RBDSmat.txt')
angh_matlab = np.loadtxt(fn_h_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(3, figsize=(10,6))
fig.suptitle('Comparison of RIGHT HIP angles between methods')
axs[0].plot(tn,angle_L_hip[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],angh_matlab[0::5,0], 'bo', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_L_hip[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],angh_matlab[0::5,1], 'ro', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_L_hip[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],angh_matlab[0::5,2], 'go', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# Import ankle angles calculated in Matlab
fn_h_matlab = os.path.join(pathname, 'r_hip_angle_RBDSmat.txt')
angh_matlab = np.loadtxt(fn_h_matlab) * (np.pi/180)

In [ ]:
fig, axs = plt.subplots(3, figsize=(10,6))
fig.suptitle('Comparison of RIGHT HIP angles between methods')
axs[0].plot(tn,angle_R_hip[:,0], 'b', label='Python')
axs[0].plot(tn[0::5],angh_matlab[0::5,0], 'bo', label='Matlab')
axs[0].grid('on')
axs[1].plot(angle_R_hip[:,1], 'r', label='Python')
axs[1].plot(tn[0::5],angh_matlab[0::5,1], 'ro', label='Matlab')
axs[1].grid('on')
axs[2].plot(angle_R_hip[:,2], 'g', label='Python')
axs[2].plot(tn[0::5],angh_matlab[0::5,2], 'go', label='Matlab')
axs[2].grid('on')
plt.show()

In [ ]:
# in case it needs to be exported to Matlab
# Marker labels
mkrs_lbls = [m[:-2] for m in neutral_RBDS.columns.tolist()]
mkrs_lbls = list(dict.fromkeys(mkrs_lbls)) # drop duplicates
# Create dictionary
neutral_dic = dict()
for mkr in mkrs_lbls:
    neutral_dic[mkr]=neutral_RBDS.filter(like=mkr, axis=1).values.flatten()
    
# Joint labels
joint_lbls = [m[:-2] for m in joints_RBDS.columns.tolist()]
joint_lbls = list(dict.fromkeys(joint_lbls)) # drop duplicates
# Create dictionary
joints_dic = dict()
for jt in joint_lbls:
    joints_dic[jt]=joints_RBDS.filter(like=jt, axis=1).values.flatten()

# Gait labels
gait_lbls = [m[:-2] for m in gait_RBDS.columns.tolist()]
gait_lbls = list(dict.fromkeys(gait_lbls)) # drop duplicates
# Create dictionary
gait_dic = dict()
for gt in gait_lbls:
    gait_dic[gt]=gait_RBDS.filter(like=gt, axis=1).values
# neutral_RBDS.to_csv(os.path.join(data_dir,'RBDS001_neutral.csv'), sep='\t')
# joints_RBDS.to_csv(os.path.join(data_dir,'RBDS001_joints.csv'), sep='\t')
# gait_RBDS.to_csv(os.path.join(data_dir,'RBDS001_gait.csv'), sep='\t')